In [1]:
from battle import *
from trainer import *
from pokemon import *
from moves import *
from globals import *
from team import *

In [2]:
moves=[Tackle(), Earthquake(), Thunder()]
pikachuStats={
    Stat.HP : 200,
    Stat.ATT : 100,
    Stat.DEF : 100,
    Stat.SPA : 100,
    Stat.SPD : 100,
    Stat.SPE : 100
}
pikachus=[Pikachu('pika1', 15, pikachuStats, moves), Pikachu('pika2', 15, pikachuStats, moves), Pikachu('pika3', 15, pikachuStats, moves), Pikachu('pika4', 15, pikachuStats, moves)]

ratStats={
    Stat.HP : 200,
    Stat.ATT : 100,
    Stat.DEF : 100,
    Stat.SPA : 100,
    Stat.SPD : 100,
    Stat.SPE : 101
}
rats=[Rattata('rat1', 20, ratStats, moves), Rattata('rat2', 20, ratStats, moves), Rattata('rat3', 20, ratStats, moves), Rattata('rat4', 20, ratStats, moves)]
t1=Trainer('Trainer 1', pikachus, None)
t2=Trainer('Trainer 2', rats, None)
teams=[Team('Team Pikachu', [t1], 3), Team('Team Rattata', [t2], 2)]
testBattle=Battle(teams)

In [3]:
testBattle.runBattle()
# TODO: sync pokemon fainted status with corresponding slot being empty
# TODO: only show trainers when swapping and sending out pokemon if they have pokemon available

List of possible trainers: 
	 0 Trainer 1
List of possible pokemon: 
	 0 pika1
	 1 pika2
	 2 pika3
	 3 pika4
pika1 was selected!


List of possible trainers: 
	 0 Trainer 1
List of possible pokemon: 
	 0 pika1
	 1 pika2
	 2 pika3
	 3 pika4
pika2 was selected!


List of possible trainers: 
	 0 Trainer 1
List of possible pokemon: 
	 0 pika1
	 1 pika2
	 2 pika3
	 3 pika4
pika3 was selected!


List of possible trainers: 
	 0 Trainer 2
List of possible pokemon: 
	 0 rat1
	 1 rat2
	 2 rat3
	 3 rat4
rat1 was selected!


List of possible trainers: 
	 0 Trainer 2
List of possible pokemon: 
	 0 rat1
	 1 rat2
	 2 rat3
	 3 rat4
rat2 was selected!


Team Pikachu
-----  -----  -----
pika1  pika2  pika3
200    200    200
-----  -----  -----


Team Rattata
----  ----
rat1  rat2
200   200
----  ----


Trainer 1 is picking a move for pika1
List of moves: 3
	 0 Tackle
	 1 Earthquake
	 2 Thunder
Thunder was selected.


Team Team Pikachu's Pokemon pika1 is choosing a target!
0 Pokemon from Team Team Pikach

1